# GoogleNet 구조

In [13]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
li = [[64, 96, 128, 16, 32, 3, 32],
    [128, 128, 192, 32, 96, 3, 64],
    [192, 96, 208, 16, 48, 3, 64],
    [160, 112, 224, 24, 64, 3, 64],
    [128, 128, 256, 24, 64, 3, 64],
    [112, 144, 288, 32, 64, 3, 64],
    [256, 160, 320, 32, 128, 3, 128],
    [256, 160, 320, 32, 128, 3, 128],
    [384, 192, 384, 48, 128, 3, 128]]
df_numbers = pd.DataFrame(li, columns = ['one', 'thr_rdc', 'thr', 'fiv_rdc', 'fiv', 'mpl', 'mpl_one'],
                  index = ['3a', '3b', '4a', '4b', '4c', '4d', '4d', '5a', '5b'])

def makeInception(inp, numbers, name):
    # inp = base layer
    # numbers = {one, thr_rdc, thr, fiv_rdc, fiv, mpl, mpl_one}
    # name = layer name for summary
    inception_1x1 = layers.Conv2D(numbers['one'], 1, strides = (1, 1), padding = 'same', name = 'inception_' + name +'_1x1/1x1_s1')(inp)
    
    inception_3x3 = layers.Conv2D(numbers['thr_rdc'], 1, strides = (1, 1), padding = 'same', name = 'inception_' + name +'_3x3/1x1_s1')(inp)
    inception_3x3 = layers.Conv2D(numbers['thr'], 3, strides = (1, 1), padding = 'same', name = 'inception_' + name +'_3x3/3x3_s1')(inception_3x3)
    
    inception_5x5 = layers.Conv2D(numbers['fiv_rdc'], 1, strides = (1, 1), padding = 'same', name = 'inception_' + name +'_5x5/1x1_s1')(inp)
    inception_5x5 = layers.Conv2D(numbers['fiv'], 5, strides = (1, 1), padding = 'same', name = 'inception_' + name +'_5x5/5x5_s1')(inception_5x5)
    
    inception_pool_proj = layers.MaxPooling2D((numbers['mpl'], numbers['mpl']), strides=1, padding = 'same', name = 'inception_' + name +'_pool_proj/3x3_sz')(inp)
    inception_pool_proj = layers.Conv2D(numbers['mpl_one'], 1, strides = (1, 1), padding = 'same', name = 'inception_' + name +'_pool_proj/1x1_s1')(inception_pool_proj)
    
    # output : concat
    return layers.Concatenate(axis=-1,
                                      name='inception_' + name +'/output')([inception_1x1,inception_3x3,
                                                                   inception_5x5,inception_pool_proj])
    
def create_googlenet1():
    numbers = {'one': 0, 'thr_rdc': 0, 'thr': 0, 'fiv_rdc': 0, 'fiv': 0, 'mpl': 0, 'mpl_one': 0}
    # creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
    input = tf.keras.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, 7, strides = (2, 2), padding = 'same', name = 'conv1/7x7_s2')(input)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool1/3x3_s2')(x)
    x = layers.Lambda(tf.nn.local_response_normalization, name = 'norm1')(x)
    x = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'conv2_1/1x1_s1')(x)
    x = layers.Conv2D(192, 3, strides = (1, 1), padding = 'same', name = 'conv2_2/3x3_s1')(x)
    x = layers.Lambda(tf.nn.local_response_normalization, name = 'norm2')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool2/3x3_s2')(x)
    # Inception 3a ############################################################
    numbers = {'one': 64, 'thr_rdc': 96, 'thr': 128, 'fiv_rdc': 16, 'fiv': 32, 'mpl': 3, 'mpl_one': 32}
    inception_3a_output = makeInception(x, numbers, '3a')
    # Inception 3b ############################################################
    numbers = {'one': 128, 'thr_rdc': 128, 'thr': 192, 'fiv_rdc': 32, 'fiv': 96, 'mpl': 3, 'mpl_one': 64}
    inception_3b_output = makeInception(inception_3a_output, numbers, '3b')
    ###########################################################################
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool3/3x3_s2')(inception_3b_output)
    
    
    # Inception 4a ############################################################
    numbers = {'one': 192, 'thr_rdc': 96, 'thr': 208, 'fiv_rdc': 16, 'fiv': 48, 'mpl': 3, 'mpl_one': 64}
    inception_4a_output = makeInception(x, numbers, '4a')
    
    # auxiliary classifier **********************
    loss1 = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss1_pool_5x5_s3')(inception_4a_output)
    loss1 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'loss1_conv/1x1_s1')(loss1)
    loss1 = layers.Flatten(name = 'loss1_flatten')(loss1)
    loss1 = layers.Dense(1024, activation='relu', name = 'loss1_relu')(loss1)
    loss1 = layers.Dropout(0.7, name = 'loss1_dropout')(loss1)
    loss1_classifier = layers.Dense(1000, activation='softmax', name = 'loss1_classifier')(loss1)
    
    # Inception 4b ############################################################
    numbers = {'one': 160, 'thr_rdc': 112, 'thr': 224, 'fiv_rdc': 24, 'fiv': 64, 'mpl': 3, 'mpl_one': 64}
    inception_4b_output = makeInception(inception_4a_output, numbers, '4b')
    # Inception 4c ############################################################
    numbers = {'one': 128, 'thr_rdc': 128, 'thr': 256, 'fiv_rdc': 24, 'fiv': 64, 'mpl': 3, 'mpl_one': 64}
    inception_4c_output = makeInception(inception_4b_output, numbers, '4c')
    # Inception 4d ############################################################
    numbers = {'one': 112, 'thr_rdc': 144, 'thr': 288, 'fiv_rdc': 32, 'fiv': 64, 'mpl': 3, 'mpl_one': 64}
    inception_4d_output = makeInception(inception_4c_output, numbers, '4d')
    
    # auxiliary classifier **********************
    loss2 = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss2_pool_5x5_s3')(inception_4d_output)
    loss2 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'loss2_conv/1x1_s1')(loss2)
    loss2 = layers.Flatten(name = 'loss2_flatten')(loss2)
    loss2 = layers.Dense(1024, activation='relu', name = 'loss2_relu')(loss2)
    loss2 = layers.Dropout(0.7, name = 'loss2_dropout')(loss2)
    loss2_classifier = layers.Dense(1000, activation='softmax', name = 'loss2_classifier')(loss2)
    
    # Inception 4e ############################################################
    numbers = {'one': 256, 'thr_rdc': 160, 'thr': 320, 'fiv_rdc': 32, 'fiv': 128, 'mpl': 3, 'mpl_one': 128}
    inception_5a_output = makeInception(inception_4d_output, numbers, '4e')
    ###########################################################################
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool4/3x3_s2')(inception_5a_output)
    # Inception 5a ############################################################
    numbers = {'one': 256, 'thr_rdc': 160, 'thr': 320, 'fiv_rdc': 32, 'fiv': 128, 'mpl': 3, 'mpl_one': 128}
    inception_4e_output = makeInception(x, numbers, '5a')
    # Inception 5b ############################################################
    numbers = {'one': 384, 'thr_rdc': 192, 'thr': 384, 'fiv_rdc': 48, 'fiv': 128, 'mpl': 3, 'mpl_one': 128}
    inception_5b_output = makeInception(inception_4e_output, numbers, '5b')
    
    ###########################################################################
    pool5_7x7_s1 = layers.AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5/7x7_s2')(inception_5b_output)
    flatten5 = layers.Flatten(name = 'flatten5')(pool5_7x7_s1)
    loss3_classifier = layers.Dropout(0.4, name = 'loss3_dropout')(flatten5)
    loss3_classifier = layers.Dense(1000, activation='softmax', name = 'loss3_classifier')(loss3_classifier)



   # Final Model : input + output[aux.cls1, aux.cls2, main.cls]
    googlenet = tf.keras.Model(inputs=input, outputs=[loss1_classifier,loss2_classifier,loss3_classifier])
    # googlenet = tf.keras.Model(inputs=input, outputs=[loss3_classifier])

    return googlenet

model = create_googlenet1()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9472        ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 pool1/3x3_s2 (MaxPooling2D)    (None, 56, 56, 64)   0           ['conv1/7x7_s2[0][0]']           
                                                                                            

In [16]:
li = [[64, 96, 128, 16, 32, 3, 32],
    [128, 128, 192, 32, 96, 3, 64],
    [192, 96, 208, 16, 48, 3, 64],
    [160, 112, 224, 24, 64, 3, 64],
    [128, 128, 256, 24, 64, 3, 64],
    [112, 144, 288, 32, 64, 3, 64],
    [256, 160, 320, 32, 128, 3, 128],
    [256, 160, 320, 32, 128, 3, 128],
    [384, 192, 384, 48, 128, 3, 128]]
numbers = pd.DataFrame(li, columns = ['one', 'thr_rdc', 'thr', 'fiv_rdc', 'fiv', 'mpl', 'mpl_one'],
                  index = ['3a', '3b', '4a', '4b', '4c', '4d', '4d', '5a', '5b'])
df

,one,thr_rdc,thr,fiv_rdc,fiv,mpl,mpl_one
3a,64,96,128,16,32,3,32
3b,128,128,192,32,96,3,64
4a,192,96,208,16,48,3,64
4b,160,112,224,24,64,3,64
4c,128,128,256,24,64,3,64
4d,112,144,288,32,64,3,64
4d,256,160,320,32,128,3,128
5a,256,160,320,32,128,3,128
5b,384,192,384,48,128,3,128


In [ ]:

def create_googlenet():
    numbers = {'one': 0, 'thr_rdc': 0, 'thr': 0, 'fiv_rdc': 0, 'fiv': 0, 'mpl': 0, 'mpl_one': 0}
    # creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
    input = tf.keras.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, 7, strides = (2, 2), padding = 'same', name = 'conv1/7x7_s2')(input)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool1/3x3_s2')(x)
    x = layers.Lambda(tf.nn.local_response_normalization, name = 'norm1')(x)
    x = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'conv2_1/1x1_s1')(x)
    x = layers.Conv2D(192, 3, strides = (1, 1), padding = 'same', name = 'conv2_2/3x3_s1')(x)
    x = layers.Lambda(tf.nn.local_response_normalization, name = 'norm2')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool2/3x3_s2')(x)
    # Inception 3a ############################################################
    inception_3a_1x1 = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'inception_3a_1x1/1x1_s1')(x)
    
    inception_3a_3x3 = layers.Conv2D(96, 1, strides = (1, 1), padding = 'same', name = 'inception_3a_3x3/1x1_s1')(x)
    inception_3a_3x3 = layers.Conv2D(128, 3, strides = (1, 1), padding = 'same', name = 'inception_3a_3x3/3x3_s1')(inception_3a_3x3)
    
    inception_3a_5x5 = layers.Conv2D(16, 1, strides = (1, 1), padding = 'same', name = 'inception_3a_5x5/1x1_s1')(x)
    inception_3a_5x5 = layers.Conv2D(32, 5, strides = (1, 1), padding = 'same', name = 'inception_3a_5x5/5x5_s1')(inception_3a_5x5)
    
    inception_3a_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_3a_pool_proj/3x3_sz')(x)
    inception_3a_pool_proj = layers.Conv2D(32, 1, strides = (1, 1), padding = 'same', name = 'inception_3a_pool_proj/1x1_s1')(inception_3a_pool_proj)
    
    # output : concat
    inception_3a_output = layers.Concatenate(axis=-1,
                                      name='inception_3a/output')([inception_3a_1x1,inception_3a_3x3,
                                                                   inception_3a_5x5,inception_3a_pool_proj])

    # Inception 3b ############################################################
    inception_3b_1x1 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'inception_3b_1x1/1x1_s1')(inception_3a_output)
    
    inception_3b_3x3 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'inception_3b_3x3/1x1_s1')(inception_3a_output)
    inception_3b_3x3 = layers.Conv2D(192, 3, strides = (1, 1), padding = 'same', name = 'inception_3b_3x3/3x3_s1')(inception_3b_3x3)
    
    inception_3b_5x5 = layers.Conv2D(32, 1, strides = (1, 1), padding = 'same', name = 'inception_3b_5x5/1x1_s1')(inception_3a_output)
    inception_3b_5x5 = layers.Conv2D(96, 5, strides = (1, 1), padding = 'same', name = 'inception_3b_5x5/5x5_s1')(inception_3b_5x5)
    
    inception_3b_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_3b_pool_proj/3x3_sz')(inception_3a_output)
    inception_3b_pool_proj = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'inception_3b_pool_proj/1x1_s1')(inception_3b_pool_proj)
    
    # output : concat
    inception_3b_output = layers.Concatenate(axis=-1,
                                      name='inception_3b/output')([inception_3b_1x1,inception_3b_3x3,
                                                                   inception_3b_5x5,inception_3b_pool_proj])

    ###########################################################################
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool3/3x3_s2')(inception_3b_output)
    # Inception 4a ############################################################
    inception_4a_1x1 = layers.Conv2D(192, 1, strides = (1, 1), padding = 'same', name = 'inception_4a_1x1/1x1_s1')(x)
    
    inception_4a_3x3 = layers.Conv2D(96, 1, strides = (1, 1), padding = 'same', name = 'inception_4a_3x3/1x1_s1')(x)
    inception_4a_3x3 = layers.Conv2D(208, 3, strides = (1, 1), padding = 'same', name = 'inception_4a_3x3/3x3_s1')(inception_4a_3x3)
    
    inception_4a_5x5 = layers.Conv2D(16, 1, strides = (1, 1), padding = 'same', name = 'inception_4a_5x5/1x1_s1')(x)
    inception_4a_5x5 = layers.Conv2D(48, 5, strides = (1, 1), padding = 'same', name = 'inception_4a_5x5/5x5_s1')(inception_4a_5x5)
    
    inception_4a_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_4a_pool_proj/3x3_sz')(x)
    inception_4a_pool_proj = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'inception_4a_pool_proj/1x1_s1')(inception_4a_pool_proj)
    
    # output : concat
    inception_4a_output = layers.Concatenate(axis=-1,
                                      name='inception_4a/output')([inception_4a_1x1,inception_4a_3x3,
                                                                   inception_4a_5x5,inception_4a_pool_proj])

    # auxiliary classifier **********************
    loss1 = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss1_pool_5x5_s3')(inception_4a_output)
    loss1 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'loss1_conv/1x1_s1')(loss1)
    loss1 = layers.Flatten(name = 'loss1_flatten')(loss1)
    loss1 = layers.Dense(1024, activation='relu', name = 'loss1_relu')(loss1)
    loss1 = layers.Dropout(0.7, name = 'loss1_dropout')(loss1)
    loss1_classifier = layers.Dense(1000, activation='softmax', name = 'loss1_classifier')(loss1)


    # Inception 4b ############################################################
    inception_4b_1x1 = layers.Conv2D(160, 1, strides = (1, 1), padding = 'same', name = 'inception_4b_1x1/1x1_s1')(inception_4a_output)
    
    inception_4b_3x3 = layers.Conv2D(112, 1, strides = (1, 1), padding = 'same', name = 'inception_4b_3x3/1x1_s1')(inception_4a_output)
    inception_4b_3x3 = layers.Conv2D(224, 3, strides = (1, 1), padding = 'same', name = 'inception_4b_3x3/3x3_s1')(inception_4b_3x3)
    
    inception_4b_5x5 = layers.Conv2D(24, 1, strides = (1, 1), padding = 'same', name = 'inception_4b_5x5/1x1_s1')(inception_4a_output)
    inception_4b_5x5 = layers.Conv2D(64, 5, strides = (1, 1), padding = 'same', name = 'inception_4b_5x5/5x5_s1')(inception_4b_5x5)
    
    inception_4b_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_4b_pool_proj/3x3_sz')(inception_4a_output)
    inception_4b_pool_proj = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'inception_4b_pool_proj/1x1_s1')(inception_4b_pool_proj)
    
    # output : concat
    inception_4b_output = layers.Concatenate(axis=-1,
                                      name='inception_4b/output')([inception_4b_1x1,inception_4b_3x3,
                                                                   inception_4b_5x5,inception_4b_pool_proj])

    # Inception 4c ############################################################
    inception_4c_1x1 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'inception_4c_1x1/1x1_s1')(inception_4b_output)
    
    inception_4c_3x3 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'inception_4c_3x3/1x1_s1')(inception_4b_output)
    inception_4c_3x3 = layers.Conv2D(256, 3, strides = (1, 1), padding = 'same', name = 'inception_4c_3x3/3x3_s1')(inception_4c_3x3)
    
    inception_4c_5x5 = layers.Conv2D(24, 1, strides = (1, 1), padding = 'same', name = 'inception_4c_5x5/1x1_s1')(inception_4b_output)
    inception_4c_5x5 = layers.Conv2D(64, 5, strides = (1, 1), padding = 'same', name = 'inception_4c_5x5/5x5_s1')(inception_4c_5x5)
    
    inception_4c_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_4c_pool_proj/3x3_sz')(inception_4b_output)
    inception_4c_pool_proj = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'inception_4c_pool_proj/1x1_s1')(inception_4c_pool_proj)
    
    # output : concat
    inception_4c_output = layers.Concatenate(axis=-1,
                                      name='inception_4c/output')([inception_4c_1x1,inception_4c_3x3,
                                                                   inception_4c_5x5,inception_4c_pool_proj])


    # Inception 4d ############################################################
    inception_4d_1x1 = layers.Conv2D(112, 1, strides = (1, 1), padding = 'same', name = 'inception_4d_1x1/1x1_s1')(inception_4c_output)
    
    inception_4d_3x3 = layers.Conv2D(144, 1, strides = (1, 1), padding = 'same', name = 'inception_4d_3x3/1x1_s1')(inception_4c_output)
    inception_4d_3x3 = layers.Conv2D(288, 3, strides = (1, 1), padding = 'same', name = 'inception_4d_3x3/3x3_s1')(inception_4d_3x3)
    
    inception_4d_5x5 = layers.Conv2D(32, 1, strides = (1, 1), padding = 'same', name = 'inception_4d_5x5/1x1_s1')(inception_4c_output)
    inception_4d_5x5 = layers.Conv2D(64, 5, strides = (1, 1), padding = 'same', name = 'inception_4d_5x5/5x5_s1')(inception_4d_5x5)
    
    inception_4d_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_4d_pool_proj/3x3_sz')(inception_4c_output)
    inception_4d_pool_proj = layers.Conv2D(64, 1, strides = (1, 1), padding = 'same', name = 'inception_4d_pool_proj/1x1_s1')(inception_4d_pool_proj)
    
    # output : concat
    inception_4d_output = layers.Concatenate(axis=-1,
                                      name='inception_4d/output')([inception_4d_1x1,inception_4d_3x3,
                                                                   inception_4d_5x5,inception_4d_pool_proj])

    # auxiliary classifier **********************
    loss2 = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss2_pool_5x5_s3')(inception_4d_output)
    loss2 = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'loss2_conv/1x1_s1')(loss2)
    loss2 = layers.Flatten(name = 'loss2_flatten')(loss2)
    loss2 = layers.Dense(1024, activation='relu', name = 'loss2_relu')(loss2)
    loss2 = layers.Dropout(0.7, name = 'loss2_dropout')(loss2)
    loss2_classifier = layers.Dense(1000, activation='softmax', name = 'loss2_classifier')(loss2)


    # Inception 4e ############################################################
    inception_4e_1x1 = layers.Conv2D(256, 1, strides = (1, 1), padding = 'same', name = 'inception_4e_1x1/1x1_s1')(inception_4d_output)
    
    inception_4e_3x3 = layers.Conv2D(160, 1, strides = (1, 1), padding = 'same', name = 'inception_4e_3x3/1x1_s1')(inception_4d_output)
    inception_4e_3x3 = layers.Conv2D(320, 3, strides = (1, 1), padding = 'same', name = 'inception_4e_3x3/3x3_s1')(inception_4e_3x3)
    
    inception_4e_5x5 = layers.Conv2D(32, 1, strides = (1, 1), padding = 'same', name = 'inception_4e_5x5/1x1_s1')(inception_4d_output)
    inception_4e_5x5 = layers.Conv2D(128, 5, strides = (1, 1), padding = 'same', name = 'inception_4e_5x5/5x5_s1')(inception_4e_5x5)
    
    inception_4e_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_4e_pool_proj/3x3_sz')(inception_4d_output)
    inception_4e_pool_proj = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'inception_4e_pool_proj/1x1_s1')(inception_4e_pool_proj)
    
    # output : concat
    inception_4e_output = layers.Concatenate(axis=-1,
                                      name='inception_4e/output')([inception_4e_1x1,inception_4e_3x3,
                                                                   inception_4e_5x5,inception_4e_pool_proj])

    ###########################################################################
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool4/3x3_s2')(inception_4e_output)
    # Inception 5a ############################################################
    inception_5a_1x1 = layers.Conv2D(256, 1, strides = (1, 1), padding = 'same', name = 'inception_5a_1x1/1x1_s1')(x)
    
    inception_5a_3x3 = layers.Conv2D(160, 1, strides = (1, 1), padding = 'same', name = 'inception_5a_3x3/1x1_s1')(x)
    inception_5a_3x3 = layers.Conv2D(320, 3, strides = (1, 1), padding = 'same', name = 'inception_5a_3x3/3x3_s1')(inception_5a_3x3)
    
    inception_5a_5x5 = layers.Conv2D(32, 1, strides = (1, 1), padding = 'same', name = 'inception_5a_5x5/1x1_s1')(x)
    inception_5a_5x5 = layers.Conv2D(128, 5, strides = (1, 1), padding = 'same', name = 'inception_5a_5x5/5x5_s1')(inception_5a_5x5)
    
    inception_5a_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_5a_pool_proj/3x3_sz')(x)
    inception_5a_pool_proj = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'inception_5a_pool_proj/1x1_s1')(inception_5a_pool_proj)
    
    # output : concat
    inception_5a_output = layers.Concatenate(axis=-1,
                                      name='inception_5a/output')([inception_5a_1x1,inception_5a_3x3,
                                                                   inception_5a_5x5,inception_5a_pool_proj])

    # Inception 5b ############################################################
    inception_5b_1x1 = layers.Conv2D(384, 1, strides = (1, 1), padding = 'same', name = 'inception_5b_1x1/1x1_s1')(inception_5a_output)
    
    inception_5b_3x3 = layers.Conv2D(192, 1, strides = (1, 1), padding = 'same', name = 'inception_5b_3x3/1x1_s1')(inception_5a_output)
    inception_5b_3x3 = layers.Conv2D(384, 3, strides = (1, 1), padding = 'same', name = 'inception_5b_3x3/3x3_s1')(inception_5b_3x3)
    
    inception_5b_5x5 = layers.Conv2D(48, 1, strides = (1, 1), padding = 'same', name = 'inception_5b_5x5/1x1_s1')(inception_5a_output)
    inception_5b_5x5 = layers.Conv2D(128, 5, strides = (1, 1), padding = 'same', name = 'inception_5b_5x5/5x5_s1')(inception_5b_5x5)
    
    inception_5b_pool_proj = layers.MaxPooling2D((3, 3), strides=1, padding = 'same', name = 'inception_5b_pool_proj/3x3_sz')(inception_5a_output)
    inception_5b_pool_proj = layers.Conv2D(128, 1, strides = (1, 1), padding = 'same', name = 'inception_5b_pool_proj/1x1_s1')(inception_5b_pool_proj)

    # output : concat
    inception_5b_output = layers.Concatenate(axis=-1,
                                      name='inception_5b/output')([inception_5b_1x1,inception_5b_3x3,
                                                                   inception_5b_5x5,inception_5b_pool_proj])

    pool5_7x7_s1 = layers.AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5/7x7_s2')(inception_5b_output)
    flatten5 = layers.Flatten(name = 'flatten5')(pool5_7x7_s1)
    loss3_classifier = layers.Dropout(0.4, name = 'loss3_dropout')(flatten5)
    loss3_classifier = layers.Dense(1000, activation='softmax', name = 'loss3_classifier')(loss3_classifier)



   # Final Model : input + output[aux.cls1, aux.cls2, main.cls]
    googlenet = tf.keras.Model(inputs=input, outputs=[loss1_classifier,loss2_classifier,loss3_classifier])
    # googlenet = tf.keras.Model(inputs=input, outputs=[loss3_classifier])

    return googlenet

In [ ]:
model = create_googlenet()

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 pool1/3x3_s2 (MaxPooling2D)    (None, 56, 56, 64)   0           ['conv1/7x7_s2[0][0]']           
                                                                                              

In [8]:
input1 = tf.keras.Input(shape=(224, 224, 3))
output1 = tf.keras.layers.Conv2D(7, 7, strides = (2, 2), padding = 'same')(input1)
m1 = tf.keras.Model(inputs=input1, outputs=output1)
m1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 7)       1036      
                                                                 
Total params: 1,036
Trainable params: 1,036
Non-trainable params: 0
_________________________________________________________________


## VGGNet 구현

In [15]:
def createVggnet():
    input = tf.keras.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, 3, strides = (1, 1), padding = 'same', name = 'conv224_1')(input)
    x = layers.Conv2D(64, 3, strides = (1, 1), padding = 'same', name = 'conv224_2')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool1')(x)
    
    x = layers.Conv2D(128, 3, strides = (1, 1), padding = 'same', name = 'conv112_1')(x)
    x = layers.Conv2D(128, 3, strides = (1, 1), padding = 'same', name = 'conv112_2')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool2')(x)
    
    x = layers.Conv2D(256, 3, strides = (1, 1), padding = 'same', name = 'conv56_1')(x)
    x = layers.Conv2D(256, 3, strides = (1, 1), padding = 'same', name = 'conv56_2')(x)
    x = layers.Conv2D(256, 3, strides = (1, 1), padding = 'same', name = 'conv56_3')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool3')(x)
    
    x = layers.Conv2D(512, 3, strides = (1, 1), padding = 'same', name = 'conv28_1')(x)
    x = layers.Conv2D(512, 3, strides = (1, 1), padding = 'same', name = 'conv28_2')(x)
    x = layers.Conv2D(512, 3, strides = (1, 1), padding = 'same', name = 'conv28_3')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool4')(x)
    
    x = layers.Conv2D(512, 3, strides = (1, 1), padding = 'same', name = 'conv14_1')(x)
    x = layers.Conv2D(512, 3, strides = (1, 1), padding = 'same', name = 'conv14_2')(x)
    x = layers.Conv2D(512, 3, strides = (1, 1), padding = 'same', name = 'conv14_3')(x)    
    x = layers.MaxPooling2D((3, 3), strides=2, padding = 'same', name = 'pool5')(x)
    
    x = layers.Flatten(name = 'flatten')(x)
    x = layers.Dense(4096, activation='relu', name = 'fc1')(x)
    x = layers.Dense(4096, activation='relu', name = 'fc2')(x)
    x = layers.Dense(4096, activation='relu', name = 'fc3')(x)
    output = layers.Dense(1000, activation='softmax', name = 'softmax')(x)

    
    
    vggnet = tf.keras.Model(inputs=input, outputs=output)

    return vggnet

model = createVggnet()
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv224_1 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv224_2 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv112_1 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv112_2 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)       0   